# Bootcamp FIAP - Turma 11IA

### Integrantes:

###    1) Fabio Rizzi - RM336656
###    2) Jurandir Ventura - RM336634
###    3) Marcelo Preto - RM336632
###    4) Thiago Alexandre – RM336583

## Desafio Bootcamp:

### Analisar o dataset de atendimentos de manutenção da empresa VIVANTE no Hospital Metropolitano e trazer insights e soluções de Machine Learning ou Deep Learning para convergir com o objetivo da empresa em liderar o mercado de Facilities com tecnologias e conhecimento disruptivos.

In [ ]:
#Importando as bibliotecas. Se houver algum erro, use "pip install Libname"
import pandas as pd
import seaborn as sb
import numpy as np
#import pydotplus
#import graphviz
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
#from sklearn.externals.six import StringIO  
from sklearn.preprocessing import MinMaxScaler,StandardScaler, OneHotEncoder
from sklearn.tree import export_graphviz, plot_tree
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, plot_confusion_matrix, mean_squared_error, r2_score, accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from xgboost import XGBClassifier
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
import tensorflow as tf
import warnings 
import np_utils
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
# Preparação do dataset
df = pd.read_excel('dataset/VIVANTE_DATASET_PIVOTADO_V1.xlsx')
print("Shape Original", df.shape)
print(list(df.columns))
print("")

In [ ]:
#Removendo linhas com estados inválidos para não estragar o modelo de previsão baseado no histórico de tempo de atividade
print("Removendo estados não finalizados")
df.drop(df[df['Estado OS'] == "AB"].index, inplace=True) # Aberto
df.drop(df[df['Estado OS'] == "CC"].index, inplace=True) # Cancelado
df.drop(df[df['Estado OS'] == "EX"].index, inplace=True) # Em Execucao
df.drop(df[df['Estado OS'] == "NE"].index, inplace=True) # Nao Executado
df.drop(df[df['Estado OS'] == "PR"].index, inplace=True) # Programada
# Removendo linhas de diálogo diário de segurança
print("Shape1:", df.shape)
df.drop(df[df['Denominação OS'] == "DDS (DIÁLOGO DIÁRIO DE SEGURANÇA)"].index, inplace=True)
print("Shape2:", df.shape)

In [ ]:
#Removendo colunas desnecessárias
df.drop(columns=['Número OS','Descrição Ativo','Estado OS','Denominação OS','Denominação Estado OS',
                 'Descrição Classe de Trabalho', 'Descrição Atividade', 'Previsão 1º Atend.',
                 'Descrição Roteiro', 'nome_tecn','Tempo Trabalho Convertido','Data da Criação',
                 'Data/Hora Abertura', 'Data Início Trabalho', 'Data Final Trabalho',
                 'Data/Hora de Conclusão', 'Roteiro'], inplace=True)
df.rename(columns={'Tempo Trabalho Feedback M.Obra': 'Tempo_Trabalhado_Real',
                    'Denominação OS': 'Denominacao_OS',
                    'Classe de Trabalho': 'Classe_Trabalho',
                    'Técnico': 'Tecnico',
                    'Tempo Previsto': 'Tempo_Previsto'}, inplace=True)
print("Colunas após removidas as desnecessárias:")
print(list(df.columns))
print("")

In [ ]:
# Removendo nulos
df.dropna(inplace=True)
print("Shape removido nulos", df.shape)
print("")

In [ ]:
df.head()

In [ ]:
df = df[['Ativo', 'Atividade', 'Classe_Trabalho', 'Tecnico', 'Tempo_Trabalhado_Real','Tempo_Previsto']]

In [ ]:
df.head()

In [ ]:
print("Shape final: ",df.shape)

In [ ]:
# Uppercase em colunas de Texto
df['Ativo']=df['Ativo'].apply(lambda x: x.upper())
df['Atividade']=df['Classe_Trabalho'].apply(lambda x: x.upper())
df['Classe_Trabalho']=df['Classe_Trabalho'].apply(lambda x: x.upper())
df['Tecnico']=df['Tecnico'].apply(lambda x: x.upper())

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df_original = df.copy()

In [ ]:
df.drop(columns = ['Tempo_Previsto'], inplace=True)

In [ ]:
df.head()

- Features: Ativo, Classe_Trabalho, Atividade, Tecnico
- Target: Tempo_Trabalhado_Real

In [ ]:
print(df['Tempo_Trabalhado_Real'].value_counts())
print("")
print( "Dataset bastante desbalanceado")

In [ ]:
value_counts = df['Tempo_Trabalhado_Real'].value_counts()
to_remove = value_counts[value_counts < 100].index
df = df[~df['Tempo_Trabalhado_Real'].isin(to_remove)]
print(df['Tempo_Trabalhado_Real'].value_counts())

In [ ]:
#y = df['Tempo_Trabalhado_Real']
#X = df.drop(columns=['Tempo_Trabalhado_Real'])

In [ ]:
scaler = StandardScaler()
y = scaler.fit_transform(df[['Tempo_Trabalhado_Real']])

In [ ]:
encoder = OneHotEncoder()
X = encoder.fit_transform(df[['Ativo','Atividade','Classe_Trabalho','Tecnico']])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=42)

In [ ]:
print("X:",X.shape)
print("Y:",y.shape)
print("X_train:",X_train.shape)
print("X_test:",X_test.shape)
print("y_train:",y_train.shape)
print("y_test:",y_test.shape)

In [ ]:
# modelo sequencial
model = Sequential()

# Input
model.add(Dense(1024, input_dim=X.shape[-1], activation='relu'))

# Hidden
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(2048, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(2048, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1024, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))

# Output
model.add(Dense(1, activation='linear'))

# Compile
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
print(model.summary())

In [ ]:
hist = model.fit(X_train, y_train, batch_size=256, epochs=200, verbose=1, shuffle=True)

In [ ]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
plt.figure(1)
plt.plot(hist.history['loss'], 'o-')
plt.plot(hist.history['mean_absolute_error'], 'x-')
plt.show()